In [1]:
import pandas as pd
import requests
from datetime import date
from sqlalchemy import create_engine, Table, MetaData, select
from time import sleep

# Receita para baixar todos os livros

Este notebook fará uma conexão com um arquivo sqlite previamente criado na mesma pasta, executa uma série de requisições para o site da Feira do Livro para baixar todo o catálogo, compila as informações e grava no sqlite.

## Conexão ao SQLite

In [2]:
db = create_engine('sqlite:///livros_usp_2024.db')

## Baixar todos os livros do site da Feira do Livro

In [3]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'priority': 'u=1, i',
    'referer': 'https://festadolivro.edusp.com.br/26-festa-do-livro-da-usp/busca-por-livros',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
}


In [4]:
pagina = 1
por_pagina = 100 # Funciona com 1000, mas recomendo usar 100 para evitar algum tipo de sobrecarga na API.

In [5]:
# Para baixar 'quase' todos os livros, vá iterando por todas as vogais. Talvez funcione também iterando pelas consoantes, mas não quis sobrecarregar a API
termo_busca = 'a'

In [6]:
params = {
    'flag': '26-festa-do-livro-da-usp',
    'page': pagina,
    'column': 'name',
    'order': 'asc',
    'per_page': por_pagina,
    'name': termo_busca,
}


In [7]:
# A primeira request é apenas para obter o número total de páginas que serão lidas
resposta = requests.get('https://festadolivro.edusp.com.br/api/v1/event-books', params=params, headers=headers)

In [ ]:
if resposta.status_code == 200:
    dados = resposta.json()['data']
    ultima_pagina = resposta.json()['meta']['last_page']
    ultima_pagina
    print('Sucesso, vamos iterar por ' + str(ultima_pagina) + ' páginas')

## Buscar livros já cadastrados
Para evitar duplicados, lemos o banco de dados e retiramos os livros já cadastrados antes.

In [9]:
livros_usp_tbl = Table('livros_usp', MetaData(), autoload_with=db)

In [10]:
select_query = select(livros_usp_tbl).order_by(livros_usp_tbl.c.publisher, livros_usp_tbl.c.authors, livros_usp_tbl.c.name)

In [11]:
with db.connect() as conn:
    livros_usp = pd.read_sql(select_query, conn)

In [ ]:
len(livros_usp) # Livros já cadastrados

In [ ]:
# O loop abaixo faz a leitura de cada página de resultados, compara os resultados encontrados com os ISBN já gravados, e salva os novos caso exista algum.
while 0 < ultima_pagina:
    params['page'] = params['page'] + 1
    resposta = requests.get('https://festadolivro.edusp.com.br/api/v1/event-books', params=params, headers=headers)
    if resposta.status_code == 200:
        resultados_df = pd.DataFrame.from_dict(resposta.json()['data'])
        resultados_df['data_atualizacao'] = pd.to_datetime(date.today())
        print('Livros encontrados: ' + str(len(resultados_df)))
        if len(resultados_df) > 0:
            with db.connect() as conn:
                livros_usp = pd.read_sql(select_query, conn)
            resultados_df = resultados_df.loc[~resultados_df['isbn'].isin(livros_usp['isbn']),].copy()
            if len(resultados_df) > 0:
                print('Livros novos para gravar: ' + str(len(resultados_df)))
                with db.connect() as conn:
                    resultados_df.to_sql('livros_usp', conn, if_exists='append', index=False)
            else:
                print('Sem livros novos para gravar')
        else:
            break
    else:
        print('Erro na página' + str(params['page']))
    sleep(5) # Quanto maior o tempo de espera em segundos, melhor para o site da Feira
    print(params['page'])